In [ ]:
import pandas as pd
import numpy as np
from m16_mlutils.pipeline import CategoryEncoder, DataFrameSelector
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from m16_mlutils.datatools.evaluation import eval_summary

from numpy.random import seed

seed(42)

In [ ]:
training_set = pd.read_csv('data/i__training_data.csv', 
                           header=None, index_col=None,
                           names=['sentence_id', 'offer_len', 'token', 'loc', 
                                  'pos', 'pos_left', 'pos_right', 
                                  'token_len', 'all_upper', 'n_tokens', 'real_label'])
training_set.head()
test_set = training_set[pd.isna(training_set['real_label'])]
training_set = training_set[~pd.isna(training_set['real_label'])]
print(len(training_set))
training_set.head()

## Load data transformation pipeline

In [ ]:
def get_pipeline():
    pipeline = joblib.load('data/i__pipeline.joblib') 
    return pipeline

## Split data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(training_set, training_set['real_label'])
print(len(X_train))
print(len(X_test))
X_test.head()

# Classify

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

pipeline = get_pipeline()

pipeline.steps.append(('classify', None))

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer


params = {
    # Different classifiers:
    'classify': [LogisticRegression(C=10), RandomForestClassifier(random_state=42), LinearSVC()],
}

grid = GridSearchCV(pipeline, cv=4, n_jobs=-1, param_grid=params, verbose=1, scoring='f1_macro')

grid.fit(X_train, y_train)

In [ ]:
print(grid.best_score_)
print()
print(grid.best_params_)

In [ ]:
estimator = grid.best_estimator_
y_pred = estimator.predict(X_test)

metrics, summary, cm = eval_summary(y_pred, y_test)

print(metrics)
print()
print(summary)

## Best estimator... for now

In [ ]:
from sklearn.base import clone

best_classifier = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)
best_pipeline = get_pipeline()
best_pipeline.steps.append(('clf', best_classifier))

In [ ]:
best_pipeline.fit(X_train, y_train)
y_pred = best_pipeline.predict(X_test)

metrics, summary, cm = eval_summary(y_test, y_pred)

print(metrics)
print(summary)

In [ ]:
joblib.dump(joint_pipeline, 'data/i__pipeline.joblib') 